In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor

In [ ]:
class DecisionTreeRegression:
    def __init__(self, criterion='squared_error', splitter='best', max_depth=None):
        self.model = DecisionTreeRegressor(criterion=criterion, splitter=splitter, max_depth=max_depth)
    def fit(self, X, y):
        self.model.fit(X, y)
        self.confidence = np.mean((y - self.predict(X)) ** 2)
    def mean_squared_error(self, y_pred, y_true):
        return np.mean((y_true - y_pred) ** 2)
    def predict(self, X):
        return self.model.predict(X)

In [ ]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None
    def fit(self, X_train, y_train):
        n_samples, n_features = X_train.shape
        self.weights = np.random.randn(n_features)
        self.bias = 0
        for i in range(self.n_iterations):
            model = np.dot(X_train, self.weights) + self.bias
            dw = (1/n_samples) * np.dot(X_train.T, (model - y_train))
            db = (1/n_samples) * np.sum(model - y_train)
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
        self.confidence = np.mean((y_train - self.predict(X_train)) ** 2)
    def predict(self, X):
        return np.dot(X, self.weights) + self.bias
    def mean_squared_error(self, y_pred, y_true):
        return np.mean((y_true - y_pred) ** 2)

In [ ]:
class LinearRegressionCustom: 
    def __init__(self, learning_rate=0.0001, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def fit(self, X_train, y_train):
        # X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]  # Add a column of ones for bias
        y_train = y_train.values.reshape(-1, 1)
        n_samples, n_features = X_train.shape

        self.weights = np.ones((n_features, 1))
        self.bias = 1

        for i in range(self.n_iterations):
            # Predictions
            predictions = np.dot(X_train, self.weights) + self.bias

            # Compute gradients
            dw = (1/n_samples) * np.dot(X_train.T, (predictions - y_train))
            db = (1/n_samples) * np.sum(predictions - y_train)

            # Update weights and bias
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            # break if any value < e-5
            if np.any(self.weights < 1e-10):
                break

    def predict(self, X):
        # X = np.c_[np.ones((X.shape[0], 1)), X]  # Add a column of ones for bias
        return np.dot(X, self.weights) + self.bias

# Example usage:
# Assuming X_train and y_train are your training data
# model = LinearRegressionCustom()
# model.fit(X_train, y_train)
# predictions = model.predict(X_test)


In [ ]:
class MultinomialLogisticRegression:
    def __init__(self, learning_rate=0.01, max_iter=1000):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.weights = []
        self.classes = None
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    def _compute_cost(self, X, y, weights):
        m = X.shape[0]
        h = self._sigmoid(X @ weights)
        cost = -1/m * np.sum(y * np.log(h + 1e-5) + (1 - y) * np.log(1 - h + 1e-5))
        return cost
    def fit(self, X, y):
        self.classes = np.unique(y)
        X = np.insert(X, 0, 1, axis=1)  
        for c in self.classes:
            y_c = (y == c).astype(int)
            weights_c = np.zeros(X.shape[1])
            for _ in range(self.max_iter):
                m = X.shape[0]
                h = self._sigmoid(X @ weights_c)
                gradient = np.dot(X.T, (h - y_c)) / m
                weights_c -= self.learning_rate * gradient
            self.weights.append(weights_c)
    def predict_proba(self, X):
        X = np.insert(X, 0, 1, axis=1)  
        probs = np.array([self._sigmoid(np.dot(X, w)) for w in self.weights]).T
        return probs / np.sum(probs, axis=1, keepdims=True)
    def predict(self, X):
        probs = self.predict_proba(X)
        return np.array([self.classes[np.argmax(p)] for p in probs])

In [2]:
class MLPRegressor: #A3
    def __init__(self, input_size, hidden_layers, learning_rate, activation, epoch):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = 1
        self.learning_rate = learning_rate
        self.neurons_layer = [input_size] + hidden_layers + [1]
        std_deviation = np.sqrt(2.0 / input_size)
        self.weights = [np.random.randn(self.neurons_layer[i], self.neurons_layer[i + 1])*std_deviation for i in range(len(self.neurons_layer) - 1)]
        self.biases = [np.zeros((1, neurons)) for neurons in self.neurons_layer[1:]]
        self.activation = activation
        self.istrain = 0
        self.epoch = epoch
        self.confidence = 1

    def fit(self, X_train, y_train):
        self.trainminibatch(X_train, y_train.values.reshape(-1, 1))
        
    def sigmoid(self, x):
        if(self.activation == 'sigmoid'):
            return 1 / (1 + np.exp(-x))
        elif(self.activation == 'tanh'):
            exp_x = np.exp(2 * x)
            tanh_x = (exp_x - 1) / (exp_x + 1)
            return tanh_x
        elif(self.activation == 'relu'):
            if self.istrain:                         # to prevent overflow we use leaky relu
                return np.maximum(0, 0.01*x)
            return np.maximum(0, x)
    def sigmoid_derivative(self, x):
        if(self.activation == 'sigmoid'):
            return x * (1 - x)
        elif(self.activation == 'tanh'):
            return 1 - x**2
        elif(self.activation == 'relu'):
            return (x > 0)
    def mean_squared_error(self, predicted, actual):
        return np.mean((predicted - actual)**2)
    def forward_propagation(self, x):
        self.layer_outputs = [x]
        for i in range(len(self.neurons_layer) - 1):
            z = np.dot(self.layer_outputs[i], self.weights[i]) + self.biases[i]
            a = self.sigmoid(z) if i != len(self.neurons_layer) - 2 else z
            self.layer_outputs.append(a)
        return self.layer_outputs[-1]
    def backward_propagation(self, X, y):
        gradients = []
        deltas = [2 * (self.layer_outputs[-1] - y)] #/ X.shape[0]]
        for i in range(len(self.neurons_layer) - 2, 0, -1):
            delta = np.dot(deltas[-1], self.weights[i].T) * self.sigmoid_derivative(self.layer_outputs[i])
            deltas.append(delta)
        deltas.reverse()
        num_samples = X.shape[0]
        for i in range(len(self.hidden_layers), -1, -1):
            grad_weights = np.dot(self.layer_outputs[i].T, deltas[i]) / num_samples
            grad_biases = np.sum(deltas[i], axis=0, keepdims=True) / num_samples
            gradients.append((grad_weights, grad_biases))
        return gradients
    def update(self, gradients):
        gradients.reverse()
        for i in range(len(self.hidden_layers) + 1):
            grad_weights, grad_biases = gradients[i]
            self.weights[i] -= self.learning_rate * grad_weights
            self.biases[i] -= self.learning_rate * grad_biases
    def train(self, X_train, y_train):
        ep = self.epoch//10
        self.istrain = 1
        for epoch in range(ep):
            for x, y in zip(X_train, y_train):
                x = x.reshape(1, -1)
                y = y.reshape(1, -1)
                self.forward_propagation(x)
                gradients = self.backward_propagation(x, y)
                self.update(gradients)
        self.istrain = 0
    def trainbatch(self, x, y):
        num_epochs = self.epoch
        for epoch in range(num_epochs):
            gradients = []
            self.forward_propagation(x)
            gradients += self.backward_propagation(x, y)
            gradients = [(np.mean(grad[0], axis=0), np.mean(grad[1], axis=0)) for grad in gradients]
            self.update(gradients)
    def trainminibatch(self, x, y):
        num_epochs = self.epoch
        for epoch in range(num_epochs):
            gradients = []
            batch_size = 50
            num_batches = x.shape[0] // batch_size
            for i in range(num_batches):
                x_batch = x[i * batch_size: (i + 1) * batch_size]
                y_batch = y[i * batch_size: (i + 1) * batch_size]
                self.forward_propagation(x_batch)
                gradients += self.backward_propagation(x_batch, y_batch)
                gradients = [(np.mean(grad[0], axis=0), np.mean(grad[1], axis=0)) for grad in gradients]
                self.update(gradients)
                gradients = []
    def predict(self, X_test):
        predictions = []
        for x in X_test:
            x = x.reshape(1, -1)
            predicted = self.forward_propagation(x)
            predictions.append(predicted)
        return np.vstack(predictions)
        

In [3]:
class MLPClassifier: #A3
    def __init__(self, input_size, hidden_layers, learning_rate, activation, epoch):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.learning_rate = learning_rate
        self.activation = activation
        self.istrain = 0
        self.epoch = epoch
        self.encoder = OneHotEncoder()

    def fit(self, X_train, y_train):
        y_temp = self.encoder.fit_transform(y_train.values.reshape(-1, 1)).toarray()
        output_size = y_temp.shape[1]
        # print(output_size)
        self.output_size = output_size
        self.neurons_layer = [self.input_size] + self.hidden_layers + [output_size]  
        self.weights = [np.random.randn(self.neurons_layer[i], self.neurons_layer[i + 1]) for i in range(len(self.neurons_layer) - 1)]
        self.biases = [np.zeros((1, neurons)) for neurons in self.neurons_layer[1:]]
        self.trainminibatch(X_train, y_temp)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / exp_x.sum(axis=1, keepdims=True)
    def sigmoid(self, x):
        if(self.activation == 'sigmoid'):
            return 1 / (1 + np.exp(-x))
        elif(self.activation == 'tanh'):
            exp_x = np.exp(2 * x)
            tanh_x = (exp_x - 1) / (exp_x + 1)
            return tanh_x
        elif(self.activation == 'relu'):
            if self.istrain:
                return np.maximum(0, 0.01*x)
            return np.maximum(0, x)
    def sigmoid_derivative(self, x):
        if(self.activation == 'sigmoid'):
            return x * (1 - x)
        elif(self.activation == 'tanh'):
            return 1 - x**2
        elif(self.activation == 'relu'):
            return (x > 0)
    def forward_propagation(self, x):
        self.layer_outputs = [x]
        for i in range(len(self.neurons_layer) - 1):
            z = np.dot(self.layer_outputs[i], self.weights[i]) + self.biases[i]
            a = self.sigmoid(z) if i < len(self.neurons_layer) - 2 else self.softmax(z) 
            self.layer_outputs.append(a)
        return self.layer_outputs[-1]
    
    def predict_proba(self, X):
        self.forward_propagation(X)
        return np.round(self.layer_outputs[-1], 5)
    
    def predict(self, X_test):
        predictions = []
        for x in X_test:
            x = x.reshape(1, -1)
            predicted = self.forward_propagation(x)
            predictions.append(predicted)
        predicted = np.vstack(predictions)
        y_temp = self.encoder.inverse_transform((predicted))
        return y_temp
    
    def backward_propagation(self, X, y):
        gradients = []
        deltas = [self.layer_outputs[-1] - y]
        for i in range(len(self.neurons_layer) - 2, 0, -1):
            delta = np.dot(deltas[-1], self.weights[i].T) * self.sigmoid_derivative(self.layer_outputs[i])
            deltas.append(delta)
        deltas.reverse()
        num_samples = X.shape[0]
        for i in range(len(self.hidden_layers), -1, -1):
            grad_weights = np.dot(self.layer_outputs[i].T, deltas[i]) / num_samples
            grad_biases = np.sum(deltas[i], axis=0, keepdims=True) / num_samples
            gradients.append((grad_weights, grad_biases))
        return gradients
    def update(self, gradients):
        gradients.reverse()
        max = 0 
        for i in range(len(self.hidden_layers) + 1):
            grad_weights, grad_biases = gradients[i]
            self.weights[i] -= self.learning_rate * grad_weights
            self.biases[i] -= self.learning_rate * grad_biases
            maxx = np.max(gradients[i][0])
            if maxx > max:
                max = maxx
            maxx = np.max(gradients[i][1])
            if maxx > max:
                max = maxx
    def train(self, X_train, y_train):
        ep = self.epoch//10
        self.istrain = 1
        for epoch in range(ep):
            for x, y in zip(X_train, y_train):
                x = x.reshape(1, -1)
                y = y.reshape(1, -1)
                self.forward_propagation(x)
                gradients = self.backward_propagation(x, y)
                self.update(gradients)
    def trainbatch(self, x, y):
        for epoch in range(self.epoch):
            gradients = []
            self.forward_propagation(x)
            gradients += self.backward_propagation(x, y)
            gradients = [(np.mean(grad[0], axis=0), np.mean(grad[1], axis=0)) for grad in gradients]
            self.update(gradients)
    def trainminibatch(self, x, y):
        for epoch in range(self.epoch):
            gradients = []
            batch_size = 50
            num_batches = x.shape[0] // batch_size
            for i in range(num_batches):
                x_batch = x[i * batch_size: (i + 1) * batch_size]
                y_batch = y[i * batch_size: (i + 1) * batch_size]
                self.forward_propagation(x_batch)
                gradients += self.backward_propagation(x_batch, y_batch)
                gradients = [(np.mean(grad[0], axis=0), np.mean(grad[1], axis=0)) for grad in gradients]
                self.update(gradients)
                gradients = []
            

In [ ]:
class MLPClassifier2: #A3
    def __init__(self, input_size, hidden_layers, learning_rate, activation, epoch):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.learning_rate = learning_rate
        self.activation = activation
        self.istrain = 0
        self.epoch = epoch
        self.encoder = OneHotEncoder()

    def fit(self, X_train, y_train):
        y_temp = self.encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
        output_size = y_temp.shape[1]
        # print(output_size)
        self.output_size = output_size
        self.neurons_layer = [self.input_size] + self.hidden_layers + [output_size]  
        self.weights = [np.random.randn(self.neurons_layer[i], self.neurons_layer[i + 1]) for i in range(len(self.neurons_layer) - 1)]
        self.biases = [np.zeros((1, neurons)) for neurons in self.neurons_layer[1:]]
        self.trainminibatch(X_train, y_temp)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / exp_x.sum(axis=1, keepdims=True)
    def sigmoid(self, x):
        if(self.activation == 'sigmoid'):
            return 1 / (1 + np.exp(-x))
        elif(self.activation == 'tanh'):
            exp_x = np.exp(2 * x)
            tanh_x = (exp_x - 1) / (exp_x + 1)
            return tanh_x
        elif(self.activation == 'relu'):
            if self.istrain:
                return np.maximum(0, 0.01*x)
            return np.maximum(0, x)
    def sigmoid_derivative(self, x):
        if(self.activation == 'sigmoid'):
            return x * (1 - x)
        elif(self.activation == 'tanh'):
            return 1 - x**2
        elif(self.activation == 'relu'):
            return (x > 0)
    def forward_propagation(self, x):
        self.layer_outputs = [x]
        for i in range(len(self.neurons_layer) - 1):
            z = np.dot(self.layer_outputs[i], self.weights[i]) + self.biases[i]
            a = self.sigmoid(z) if i < len(self.neurons_layer) - 2 else self.softmax(z) 
            self.layer_outputs.append(a)
        return self.layer_outputs[-1]
    
    def predict_proba(self, X):
        self.forward_propagation(X)
        return np.round(self.layer_outputs[-1], 5)
    
    def predict(self, X_test):
        predictions = []
        for x in X_test:
            x = x.reshape(1, -1)
            predicted = self.forward_propagation(x)
            predictions.append(predicted)
        predicted = np.vstack(predictions)
        y_temp = self.encoder.inverse_transform((predicted))
        return y_temp
    
    def backward_propagation(self, X, y):
        gradients = []
        deltas = [self.layer_outputs[-1] - y]
        for i in range(len(self.neurons_layer) - 2, 0, -1):
            delta = np.dot(deltas[-1], self.weights[i].T) * self.sigmoid_derivative(self.layer_outputs[i])
            deltas.append(delta)
        deltas.reverse()
        num_samples = X.shape[0]
        for i in range(len(self.hidden_layers), -1, -1):
            grad_weights = np.dot(self.layer_outputs[i].T, deltas[i]) / num_samples
            grad_biases = np.sum(deltas[i], axis=0, keepdims=True) / num_samples
            gradients.append((grad_weights, grad_biases))
        return gradients
    def update(self, gradients):
        gradients.reverse()
        max = 0 
        for i in range(len(self.hidden_layers) + 1):
            grad_weights, grad_biases = gradients[i]
            self.weights[i] -= self.learning_rate * grad_weights
            self.biases[i] -= self.learning_rate * grad_biases
            maxx = np.max(gradients[i][0])
            if maxx > max:
                max = maxx
            maxx = np.max(gradients[i][1])
            if maxx > max:
                max = maxx
    def train(self, X_train, y_train):
        ep = self.epoch//10
        self.istrain = 1
        for epoch in range(ep):
            for x, y in zip(X_train, y_train):
                x = x.reshape(1, -1)
                y = y.reshape(1, -1)
                self.forward_propagation(x)
                gradients = self.backward_propagation(x, y)
                self.update(gradients)
    def trainbatch(self, x, y):
        for epoch in range(self.epoch):
            gradients = []
            self.forward_propagation(x)
            gradients += self.backward_propagation(x, y)
            gradients = [(np.mean(grad[0], axis=0), np.mean(grad[1], axis=0)) for grad in gradients]
            self.update(gradients)
    def trainminibatch(self, x, y):
        for epoch in range(self.epoch):
            gradients = []
            batch_size = 50
            num_batches = x.shape[0] // batch_size
            for i in range(num_batches):
                x_batch = x[i * batch_size: (i + 1) * batch_size]
                y_batch = y[i * batch_size: (i + 1) * batch_size]
                self.forward_propagation(x_batch)
                gradients += self.backward_propagation(x_batch, y_batch)
                gradients = [(np.mean(grad[0], axis=0), np.mean(grad[1], axis=0)) for grad in gradients]
                self.update(gradients)
                gradients = []
            